<a href="https://colab.research.google.com/github/InTarsis/GMotion/blob/main/GMotion_Test_site_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://site.unibo.it/rawmatcop-alliance/en/@@images/aa4c85c2-ecf6-48e4-a412-d3b49dc48ae7.png)


<style>
body {
  font-family: Arial, sans-serif;
  line-height: 1.6;
}

h1, h2 {
  color: #333;
}

h1 {
  border-bottom: 1px solid #3B8A4C;
  padding-bottom: 10px;
  margin-bottom: 20px;
}

h2 {
  margin-top: 40px;
}

code {
  font-family: Consolas, monospace;
  font-size: 14px;
  background-color: #f9f9f9;
  border: 1px solid #ccc;
  padding: 5px;
}

.graph-image {
  max-width: 100%;
  height: auto;
  margin-bottom: 20px;
}

.module-overview {
  margin-top: 40px;
  margin-bottom: 20px;
  font-weight: bold;
}

.module-overview p {
  margin-bottom: 10px;
}
.disclaimer {
  font-family: 'Courier New', Courier, monospace;
  font-size: 14px;
  color: #555;
  margin-top: 40px;
  background-color: #f5f5f5;
  border-left: 4px solid #888;
  padding: 10px;
  line-height: 1.4;
}

</style>
<a name="Title">
  <hr color="#3B8A4C">
  <font size="7" color="#3B8A4C">GMotion: Test site</font>
  <hr color="#3B8A4C">
</a>
Module Overview
<div class="module-overview">
  <p><strong>Objective:</strong> Learne to postprocess the ground motion data from the Copernicus EGMS platform to visualice timeseries maps and perform timeseries cross-sections</p>
  <p><strong>Date:</strong> 1 Feb 2024</p>
  <p><strong>Instructors:</strong> Ignacio Marzan</p>
  <p><strong>Material and code developer:</strong> Ignacio Marzan</p>
   <p><strong>Collaborators:</strong> Elsy Ibrahim, Sara Kasmaeeyazdi, Louis Andreani, Thorkild Rasmussen, Christian Koehler</p>
  <p><strong>Contact:</strong> i.marzan@csic.es</p>
</div>
<div style="text-align: center;">







<div class="disclaimer">
  <p>
    <strong>Disclaimer:</strong>
    <br>
    <code>
      This collaborative notebook is provided for educational purposes only. The code and information shared within this notebook should not be used for any production or commercial purposes without proper validation and understanding.
    </code>
  </p>
  <p>
    <code>
      If you use or refer to the code from this notebook, please attribute it to the original authors and provide a link or reference to this notebook that you can find in the Github readme file (https://github.com/InTarsis/GMotion.git)
    </code>
  </p>
</div>

### <a name="Section 1"><hr color="#3B8A4C"><font size="6" color="#3B8A4C"> Background</font><hr color="#3B8A4C"></a>


Case study: Test site

The Copernicus European Ground Motion Service ([EGMS](https://egms.land.copernicus.eu/)) is based on the multi-temporal interferometric analysis of Sentinel-1 radar images at full resolution. It provides unprecedented resolution and coverage to study geohazards and human-induced deformation such as slow-moving landslides, subsidence due to groundwater exploitation or underground mining, volcanic unrests and many more.


**Description**:  This R script is dedicated to post_process the ground motion data from the Copernicus EGMS platform. The code reads the EGMS files, subsets the data according to specified region of interest ROI, and
generates interactive maps and plots to conduct timeseries analysis.
The main products are timeseries multilayer maps and timeseries cross-sections.

The files downloaded from EGMS are in fixed extension tiles. These files are large in size, especially the calibrated products, and it is worth noting that handling these files can consume significant resources. To improve efficiency, the script uses zip files without decompressing them.

The main parameters that need to be modified to adapt to the specifics of the case study are as follows:
- Section 2.2 .... ROI: adjust the coordinate limits to the study area.
- Section 3.1 .... *grid_step*: spacing in meters of the interpolation grid. Attention, very fine grids increase computation time.
- Section 3.4 .... Outliers: In case of identifying outliers that may affect the results, assign values to *vel_low* and *vel_up*.
- Section 3.5.3 .. Masking: Interpolation in areas devoid of data is not reliable. Assign a value to *offset* as a confidence distance for interpolation.
- Section 4.2 .... Color palette: Select minimum and maximum velocity values for *darkred* and *darkblue*. Maintain symmetry so that *green* corresponds with zero.
- Section 5.1 .... Choose the time period for the analysis of timeseries.
- Section 5.5 .... Outliers: In case of identifying outliers that may affect the results, select layers to act on and assign values to *displ_low* and *disp_up*. Repeat the process as necessary.
- Section 6.2 .... Color palette: Select minimum and maximum displacement values for *darkred* and *darkblue*. Maintain symmetry so that *green* corresponds with zero.
- Section 7.1 .... Select cross-section coordinates from the line tool on the dynamic map.

Attention!
Before you begin, ensure that you have switched your Colab environment to R using the menu:
Runtime > Change runtime type > R

In [ ]:
#
#------- GMotion -----------------------------------
#
# Version:      1.1
# Project Name: RMC-Alliance
# Author:       i.marzan@csic.es
# Date:         Feb 2024
# License:      CC BY-NC
#
# Case study:   Test site
#
# Description:  This R script is dedicated to post_process the ground motion data from the Copernicus EGMS platform.
#               The code reads the EGMS files, subsets the data according to specified region of interest ROI, and
#               generates interactive maps and plots to conduct timeseries analysis.
#               The main products are timeseries multilayer maps and timeseries cross-sections.
#
# Comment: This code can be applied to any zip file downloaded from the Copernicus EGMS platform. Optionally,
#          you can find data of prepared case studies in this repository https://github.com/InTarsis/EGMSdata.git

#          The files downloaded from EGMS are in fixed extension tiles. These files are large in size,
#          especially the calibrated products, and it is worth noting that handling these files can consume significant resources.
#          To improve efficiency, the script uses zip files without decompressing them.

#          The main parameters that need to be modified to adapt to the specifics of the case study are as follows:
#          Section 2.2 .... ROI: adjust the coordinate limits to the study area.
#          Section 3.1 .... grid_step: spacing in meters of the interpolation grid. Attention, very fine grids increase computation time.
#          Section 3.4 .... Outliers: In case of identifying outliers that may affect the results, assign values to vel_low and vel_up.
#          Section 3.5.3 .. Masking: Interpolation in areas devoid of data is not reliable. Assign a value to offset as a confidence distance for interpolation.
#          Section 4.2 .... Color palette: Select minimum and maximum velocity values for darkred and darkblue. Maintain symmetry so that green corresponds with zero.
#          Section 5.1 .... Choose the time period for the analysis of timeseries.
#          Section 5.5 .... Outliers: In case of identifying outliers that may affect the results, select layers to act on and assign values to displ_low and disp_up. Repeat the process as necessary.
#          Section 6.2 .... Color palette: Select minimum and maximum displacement values for darkred and darkblue. Maintain symmetry so that green corresponds with zero.
#          Section 7.1 .... Select cross-section coordinates from the line tool on the dynamic map.
#
# cite:         Use the Zenodo doi available in the Github readme (https://github.com/InTarsis/GMotion.git)
# -----------------------------------------------------
#

### <a name="Section 2"><hr color="#3B8A4C"><font size="6" color="#3B8A4C">**1. Notebook setup**</font><hr color="#3B8A4C"></a>


The aim of this section is to setup the script notebook. We will install the required packages, which are not already preinstalled in Colab, for the subsequent analyses. Then we will import it.

The workflow implemented in this notebook requires the following libraries:

• [tidyverse](https://www.tidyverse.org/packages/) For data manipulation and visualization.

• [scales](https://scales.r-lib.org/reference/index.html) For axis labeling, data scaling, and defining color palettes for enhanced data visualization.

• [akima](https://www.rdocumentation.org/packages/akima/versions/0.6-3.4) For interpolation of irregularly and regularly spaced data.

• [terra](https://rspatial.org/spatial/1-introduction.html) Next-generation interface to spatial data in R, based on the raster package.

• [leaflet](https://rstudio.github.io/leaflet/) For creating interactive maps within R.

• [leaflet.extras](https://bhaskarvk.github.io/leaflet.extras/) Leaflet extra functionality

• [htmlwidgets](https://www.htmlwidgets.org/) To transform graphic product into a HTML file.


In [ ]:
###############
# 1. packages
#

Pacakages: The installation of some of the next packages can take several minutes in Colab.

In [ ]:
#' First we install some packages from the Github repository that require advanced functionalities

# Remotes is a package that allows access to development packages
if (!require(remotes)) {install.packages('remotes')}

if (!require(terra)) {remotes::install_github('rspatial/terra')} # At least version 1.7.72 of terra is required
if (!require(akima)) {remotes::install_github('cran/akima')}

In [ ]:
# Installing packages from CRAN respository
if (!require(tidyverse)) {install.packages("tidyverse")}
if (!require(leaflet)) {install.packages("leaflet")}
if (!require(leaflet.extras)) {install.packages("leaflet.extras")}

Import the libraries for this session

In [ ]:
# loading installed packages into the session
library(tidyverse)   # For data manipulation and visualization
library(scales)      # To adjust values out of range to the color palette
library(akima)       # For interpolation of irregularly and regularly spaced data
library(terra)       # Next-generation interface to spatial data in R, based on the raster package
library(leaflet)     # For creating interactive maps within R
library(leaflet.extras) # Leaflet extra functionality
library(htmlwidgets)  # To transform graphic product into a HTML file

#<a name="Section 3"><hr color="#3B8A4C"><font size="6" color="#3B8A4C">**2. Input data exploration**</font><hr color="#3B8A4C"></a>


The aim of this section is to import, explore and prepare the required dataset.

In [ ]:
###################
### 2. INPUT DATA ###
#The aim of this section is to import, explore and prepare the required dataset.

#### <a name="Section 3.1"><font size="5" color="#1F497D">Import data from github repository </font></a>

Access to a data repository on Github

In [ ]:
# 2.1 Import EGMS data
#
# Clone data file
system("git clone https://github.com/InTarsis/EGMSdata.git")

Data file path

In [ ]:
# EGMS data file including region of interest (ROI)

             # Bologna water extraction subsidence -> Ortho vertical 2015-2021 and 2018-2022
             # https://www.esa.int/ESA_Multimedia/Images/2022/07/Subsidence_patterns_around_Bologna
#EGMS_zip = "/content/EGMSdata/EGMS_Bologna/EGMS_L3_E44N23_100km_U_2015_2021_BLQ.zip"
#EGMS_zip = "/content/EGMSdata/EGMS_Bologna/EGMS_L3_E44N23_100km_U_2018_2022_BLQ.zip"

             # Guadalentin water extraction subsidence -> Ortho vertical 2018-2022
             # https://doi.org/10.1016/j.enggeo.2015.08.014
#EGMS_zip = "/content/EGMSdata/EGMS_Guadalentin/subset_EGMS_L3_E32N16_100km_U_2018_2022_1_Guadalentin.zip"

             # Oelsnitz-Erzgebirge mining uplift 2018-2022
             # https://doi.org/10.3390/mining1010004
#EGMS_zip = "/content/EGMSdata/EGMS_Oelsnitz-Erzgebirge/EGMS_L3_E45N30_100km_U_2018_2022_Oelsnitz-Erzgebirge.zip"                # Ortho vertical
#EGMS_zip = "/content/EGMSdata/EGMS_Oelsnitz-Erzgebirge/subset_EGMS_L2b_168_0772_IW1_VV_2018_2022_Oelsnitz-Erzgebirge_desc.zip"  # Calibrated desc.

             # Ruhr minig subsidence 2018-2022
             # https://www.esa.int/ESA_Multimedia/Images/2020/11/Subsidence_in_the_Ruhr_Germany
#EGMS_zip = "/content/EGMSdata/EGMS_Ruhr/EGMS_L3_E40N30_100km_U_2018_2022_Ruhr.zip"                # Ortho vertical
#EGMS_zip = "/content/EGMSdata/EGMS_Ruhr/subset_EGMS_L2b_139_0772_IW3_VV_2018_2022_Ruhr_desc.zip"  # Calibrated desc.

In [ ]:
# List files inside .zip without unzip it
zip_list = unzip(EGMS_zip, list = TRUE)
zip_list

#### <a name="Section 3.1"><font size="5" color="#1F497D">Subseting the EGMS file to our ROI </font></a>

EGMS data is served in tiles that span large areas. Our region of interest (ROI) is a small part of one of these tiles, which we need to clip. In fact, it falls right on the boundary of the tile, and if we wanted to conduct a more comprehensive study of the observed deformation, we could download neighboring tiles, clip them, and connect them.

To subset our tile contained in the .zip file, we set some boundaries. In this case, the boundaries we propose cover an area of interest that extends to the north and west of Bologna, and is larger than the one contained in the file. The reason for this is that we will be able to use it to clip the neighboring tiles.

In [ ]:
## 2.2 Subseting
#     Subseting is require as Our region of interest (ROI) is a small part of original EGMS
#
# The EGMS projection is in EPSG:3035 ETRS89-extended / LAEA Europe / Lambert azimuthal equal-area projection
# limits to the ROI in meters
W_limit = 4365000 ; E_limit = 4450000
S_limit = 2375000 ; N_limit = 2420000

To apply the subsetting function without unzip the file, we need the name of the file to be clipped that is inside the zip. It's the only one with the same name but with the .csv extension.

In [ ]:
# get csv file name inside the zip file
EGMS_csv <- subset(zip_list, grepl(".csv$", Name))
EGMS_csv

In [ ]:
# Subsetting csv file without unzip it using unz(zip_file, working_file_name)
egms = subset(read_csv(unz(EGMS_zip, EGMS_csv$Name)),
             easting  < E_limit & easting  > W_limit &
             northing < N_limit & northing > S_limit)

#### <a name="Section 3.1"><font size="5" color="#1F497D">Exploring the subset EGMS dataset </font></a>

In this section, we are going to explore the EGMS dataset. We will see what type of object it is, what its dimensions are, how it is structured.

In [ ]:
# 2.3 Exploring the subset dataset
class(egms)
paste("number of rows", dim(egms)[1], ", number of columns", dim(egms)[2])

Header of the columns up to the first timeseries, which contains the initial deformation value.

In [ ]:
#     To visualize columns up until the beginning of the timeseries data:
#     Using the "seasonality_std" column number, which is just before the start of the timeseries data.
      timeseries_init <- match("seasonality_std", names(egms)) + 1
#
#     Columns up to the first value of the time series
      glimpse(egms[, 1:(timeseries_init)])

The statistics of some columns, and the distribution graph of mean velocity values:

In [ ]:
# Statistics for coordinates and mean_velocity
summary(egms[, c("easting", "northing", "mean_velocity")])

In [ ]:
# Define variables for labeling graphs based on the code in the EGMS datafile name.
vertical   = "Upwards"
horizontal = "Eastward"
LOS        = "LOS"
#
# Assign variable based on file name
if (grepl("_U.", EGMS_zip)) {
  file_type <- vertical
} else if (grepl("_E.", EGMS_zip)) {
  file_type <- horizontal
} else if (grepl("_IW", EGMS_zip)) {
  file_type <- LOS
}

# Label for velocity plots
vel_label  = paste(file_type, "mean-vel mm/y")
# Label for displacement plots
disp_label = paste(file_type, "displacement mm")

In [ ]:
#' Mean velocity Data distribution
boxplot(egms$mean_velocity, main = vel_label)

#<a name="Section 4"><hr color="#3B8A4C"><font size="6" color="#3B8A4C">**3. Mean-velocity data interpolation**</font><hr color="#3B8A4C"></a>

The aim of this section is to map the mean-velocity data, explore its spatial distribution, and identify the main features.

#### <a name="Section 3.1"><font size="5" color="#1F497D">Building a mean-velocity raster </font></a>

The first step is to build the raster is build a grid to nest the interpolation values. Change the grid spacing at your convenience, starting with coarse spacing in the first iterations because finer grids can be time-consuming.

In [ ]:
# 3.1. Build the grid to be used for nesting the interpolation values

# Set the grid spacing in meters
grid_step <- 400

# Create x and y values of our grid using a spacing of grid_step
x_cells <- with(egms, seq(min(easting), max(easting), by = grid_step))
y_cells <- with(egms, seq(min(northing), max(northing), by = grid_step))

Then we apply a spline interpolation of the egms values to de grid nodes.

In [ ]:
# 3.2. Perform spline interpolation on grid nodes from mean_velocity values
egms_vel <- with(egms, interp(x = easting, y = northing, z = mean_velocity,
                 xo = x_cells, yo = y_cells,
                 duplicate = "mean", linear=TRUE)) # if linear=False cubic spline is computed
class(egms_vel)

[1] "list"

In [ ]:
# 3.3. Rastering. Convert the interpolation result to a raster
       egms_vel_r <- rast(egms_vel)

#      define raster reference system LAEA Europe
       crs(egms_vel_r) <- "EPSG:3035"

       egms_vel_r    # raster summary

#      Plot the raster
       plot(egms_vel_r, main = vel_label)

In [ ]:
#' Raster data distribution
   boxplot(egms_vel_r, main = vel_label) #' Data distribution

ERROR: Error in eval(expr, envir, enclos): object 'egms_vel_r' not found


#### <a name="Section 3.1"><font size="5" color="#1F497D">Cleaning the mean-velocity raster ourliers</font></a>

This step is optional.

In some areas, interpolation can produce mathematical artifacts (outliers), especially at dataset boundaries, affecting the results. Apply this lines if you have identified outliers to eliminate in the boxplot.

In [ ]:
# 3.4. Outliers: In some areas, interpolation can produce mathematical artifacts (outliers),
#'               especially at the boundaries, affecting the results.
#'               If you have identified outliers through the boxplot or other means,
#'               use these commands to remove them from the raster.

#'     Optional: In case you identify some velocity values as limits for outliers,
#'               use this command to replace them with NA.
#'               Modify lower  and  upper limits accordingly.
                    vel_low = NA ; vel_up = NA
#'
#'               Replace wiht NA the values out fo range
                 egms_vel_r[egms_vel_r < vel_low | egms_vel_r > vel_up ] <- NA

#                Raster data distribution
                 boxplot(egms_vel_r, main = vel_label)

ERROR: Error: object 'egms_vel_r' not found


#### <a name="Section 3.1"><font size="5" color="#1F497D">Masking areas with no data</font></a>

The interpolation results are not valid in areas with no data.
This areas need to be masked.

In [ ]:
#' 3.5. The interpolation results are not valid in areas with no data.
#'    This areas need to be masked.

Plot the EGMS points on the raster to see where the uncovered areas are.

In [ ]:
#     3.5.1 Get egms coordinates file with the LAEA Europe coordinates reference system (CRS). Coordinates are in meters.
      egms_vel_v <- vect(cbind(egms$easting, egms$northing), crs = "EPSG:3035") # SpatVector object of terra package

      egms_vel_v # summary

      # Plot the raster
      plot(egms_vel_r, main = vel_label)
      points(egms_vel_v, cex = .3) # Plot the egms values on the raster

Now compute the distance from each raster cell to the nearest EGMS point to obtain a threshold parameter with which to mask low-density areas.

In [ ]:
#     3.5.2 Now compute distances between for each raster cell to the near egms point
      distance_rv <- distance(egms_vel_r, egms_vel_v)
      distance_rv

Masking areas with points further away than the offset value.

In [ ]:
#     3.5.3 Masking
          # preserve the original raster, work on the copy
           egms_vel_r2 = egms_vel_r # preserve the original raster, work on the copy

      #    Determine the maximum distance in meters for interpolation confidence
           offset = 700

      #    Change all values in egms_vel_r2 to NA if the corresponding distance in distance_rv is greater than x
           egms_vel_r2[distance_rv > offset] <- NA # distance in meters, choose the offset.

      plot(egms_vel_r2, main = vel_label)
      points(egms_vel_v, cex = .3)

#<a name="Section 5"><hr color="#3B8A4C"><font size="6" color="#3B8A4C">**4. Plotting mean-velocity on a dynamic map**</font><hr color="#3B8A4C"></a>

In this section, we are going to plot the mean velocity raster on a dynamic map with reference locations with which we can interact.

In [ ]:
#'##############
### 4. PLOTTING MEAN VELOCITY ON DYNAMIC MAP
#'##############

In [ ]:
# 4.1 Project the raster to a geographic reference system.
# Global dynamic maps commonly use geographic coordinates, meaning they are expressed in degrees.
egms_vel_rd = project(egms_vel_r2, "EPSG:4326")

Create a palette with the InSAR color convention

In [ ]:
# 4.2 Color palette
#  Define a color ramp with the InSAR color convention and scale limits
   egms_colors = c("darkred", "red", "orange", "yellow", "green","cyan", "dodgerblue", "blue", "darkblue")
   zlim_v <- c(-25, 25) # mm/y velocity limits
#
#' Create a color palette with colorNumeric function
#' using the previously defined variables egms_colors and zlim_v
   mypal_vel <- colorNumeric(palette = egms_colors, # color gradient to use
                          domain = zlim_v,       # range of the numeric variable
                          na.color = NA          # no color for NA values
                          )


Adjust values out of range to the color palette

In [ ]:
#' 4.3 Function to adjust values out of range to the color palette
      adjust_values <- function(x, range = zlim_v) {
                                squish(x, range)
                                                   }
#     Apply the function to the raster to be plot
      egms_vel_plot <- adjust_values(egms_vel_rd)

We use the leaflet library in combination with the terra library to create the dynamic and interactive map

In [ ]:
# 4.3 Create a dynamic map with the lonlat (degrees) raster and a custom color palette
   p_vel <- plet(egms_vel_plot,       # raster file
              alpha=0.7,              # opacity
              tiles="Esri.WorldImagery", # basemap provider, more options: https://leaflet-extras.github.io/leaflet-providers/preview/
              col = mypal_vel,        # color palette
              legend = NULL           # no legend
             )  %>%
         addLegend(
              position = "bottomleft",  # add a legend
              pal = mypal_vel,          # legend colors
              values = zlim_v,          # legend values
              title = vel_label,        # legend title
              opacity = 0.7             # legend opacity
             )  %>%
         addScaleBar("bottomright"      # add scalebar
             ) %>%
         addDrawToolbar(                # add toolbar
                        polylineOptions = TRUE,   # Enable polyline draw
                        polygonOptions      = FALSE,
                        circleOptions       = FALSE,
                        circleMarkerOptions = FALSE,
                        markerOptions       = FALSE,
                        rectangleOptions    = FALSE,
             )%>%
         addReverseSearchOSM(           # add location to polyline nodes
                        showSearchLocation = TRUE,
                        showFeature = FALSE,
                        fitBounds   = FALSE,
                        displayText = FALSE
             )

To visualice the map generate a html in your files folder, download it to your computer and open it in a browser. On the map, you can see the main features, a large subsidence pattern to the northwest of Bologna and a smaller one in Budrio. Additionally, we have created a tool to draw lines and extract coordinates that we will use later to create cross-sections.

In [ ]:
# 4.4 Transform the plot into a HTML file
 saveWidget(p_vel, "Mean_velocity.html")

ERROR: Error in saveWidget(p_vel, "Mean_velocity.html"): could not find function "saveWidget"


#<a name="Section 6"><hr color="#3B8A4C"><font size="6" color="#3B8A4C">**5. Mapping ground motion timeseries**</font><hr color="#3B8A4C"></a>

In this section, we are going to apply the mapping we have done with velocity to the ground deformation time series.

In [ ]:
#'##############
### 5. MAPPING GROUND MOTION TIMESERIES
#'##############

#### <a name="Section 3.1"><font size="5" color="#1F497D">Building a timeseries multilayer raster </font></a>

We can apply it to all time series, which are in the hundreds, but we will skip some series to reduce the computation time. We could also select a specific period if necessary.

In [ ]:
# 5.1. Define the timeseries monitoring period
#'
#'  Number of timeseries layers
    paste('The egms file has', ncol(egms) - timeseries_init, 'timeseries layers')
#'  Note: The interpolation process over a large number of layers can be time-consuming.

#'  Select the layer to work on.
#   Atention! The interpolation over a large number of layers can be time-consuming.
#'  Specify: first and last layers and the sequential increment
    first_col = timeseries_init  # timeseries_init is the column where the timeseries begins (see section 2.3)
    last_col  = ncol(egms)
    leap_col  = 10   # layer number increment, the lower the number the greater the number of layers

    timeseries = colnames(egms)[seq(first_col, last_col, by = leap_col)]

ERROR: Error in eval(expr, envir, enclos): object 'egms' not found


Create the GMotion interpolation function to be applied to the selected timeseries columns, in the same way as it was done with the mean velocity in section 3.

In [ ]:
# 5.2.Interpolation: Create the function GMotion to apply interpolation to the ground motion timeseries.
    #    GMotion takes the selected monitoring period, performs the akima interpolation for each timeseries column,
    #    using the same grid created in section 3.1, and returns a raster list object.
#
   GMotion <- function(timeseries) {
      # Perform the interpolation using the same grid define in section 3
      grd <- with (egms, akima::interp(x = easting, y = northing, z = timeseries,
                                   xo = x_cells, yo = y_cells,
                                   duplicate = "mean", linear = TRUE)) # if linear=False cubic spline is computed
      r = rast(grd)
      return(r)
      }

Apply GMotion to each of the selected timeseries. Attention! Depending on the resolution of the grid and the number of time series, this operation may take several minutes.

In [ ]:
# 5.3. Apply the GMotion function to each column and store the results in a list.
#
#    Attention!
#    This operation can take several minutes depending on the parameters: grid_step & leap_col
     egms_ts <- lapply(egms[timeseries], GMotion)

#    The result is a list of rasters
     paste("The result is a", class(egms_ts), "of", length(egms_ts) , "rasters")

[1] "The result is a list of 31 rasters"

In [ ]:
f# 5.4. Stack the raster list into a multi-layer raster object

     egms_ts_r <- rast(egms_ts)

     # Apply LAEA Europe reference system , then coordinates are in meters
     crs(egms_ts_r) <- "EPSG:3035"

     egms_ts_r      # raster summary

     boxplot(egms_ts_r, main = disp_label)  #' Data distribution

#### <a name="Section 3.1"><font size="5" color="#1F497D">Cleaning and masking unreliable values</font></a>

As in sections 3.2 and 3.3, we will clean outliers and mask areas with low data density.

In [ ]:
# 5.5. Outliers: This step is optional
#               Just as we did in section 3.4, we are going to eliminate outliers
     #'         that you may have identified through the boxplot or other means.
     #'         In this case, we have a new Raster that stacks the timeseries layers.
     #'         Define the layer to work upon, the limit displacement values and run the loop.

     # Optional: Define a vector with the number of layers you want to modify.
                nlayers = c(20, 25:31) #' These values are only provided as examples;
                                       #  modify them as appropriate.

     # Optional: Modify lower  and  upper limits accordingly.
                disp_low = NA ; disp_up = NA

     # Optional: Loop to apply limit displacement values to each layer
                 for (i in seq_along(nlayers)) {
                   n = nlayers[i]
                   layer <- egms_ts_r[[n]]

                   # Replace values outside these limits with NA.
                   layer[layer < disp_low | layer > disp_up] <- NA

                   # Save the adjusted layer back into the Raster
                   egms_ts_r[[n]] <- layer
                 }

     #  Data distribution
     boxplot(egms_ts_r , main = disp_label)

In [ ]:
# 5.6. Masking: Masking areas with no data (see section 3.5)
     egms_ts_r2 = egms_ts_r # copy raster

#    offset variable is the maximum distance for interpolation confidence (see section 3.5.3)
#    Change all values in egms_ts_r2 to NA if the corresponding distance in distance_rv is greater than x
     egms_ts_r2[distance_rv > offset] <- NA

#<a name="Section 6"><hr color="#3B8A4C"><font size="6" color="#3B8A4C">**6. Plotting timeseries on a dynamic map**</font><hr color="#3B8A4C"></a>

As in section 4, we are going to plot the timeseries raster on a dynamic map with which we can interact. The timeseries layers can be displayed using the icon in the top right corner.

In [ ]:
#'##############
### 6. PLOTTING TIME SERIES OVER DYNAMIC MAP
#'##############

Project to geographic coordinates as required by dynamic maps

In [ ]:
# 6.1 Project the raster to a geographic reference system, coordinates are in degrees.
egms_ts_rd = project(egms_ts_r2, "EPSG:4326")

Create color palette for the timeseries plot

In [ ]:
# 6.2 Define the limits of the color palette for displacement data
zlim_disp <- c(-150, 150)

# Create a colorNumeric object "mypal_disp"
mypal_disp <- colorNumeric(palette = egms_colors, domain = zlim_disp, na.color = NA)

Adjust values out of range to the color palette

In [ ]:
#' 6.3 Function to adjust values out of range to the color palette
adjust_values <- function(x, range = zlim_disp) {
                         squish(x, range)
                                             }
#     Apply the function to the raster to be plot
egms_ts_plot <- adjust_values(egms_ts_rd)

Use the leaflet library in combination with the terra library to create the dynamic and interactive map

In [ ]:
# 6.4 Plot timeseries on a dynamic base map
p_ts <- plet(egms_ts_plot,                 # raster file
              y = c(1:nlyr(egms_ts_rd)),   # raster layers
              alpha=0.7,                   # opacity
              tiles="Esri.WorldImagery",      # basemap provider, more options: https://leaflet-extras.github.io/leaflet-providers/preview/
              col = mypal_disp,            # color palette
              legend = NULL,               # no legend
              ) %>%
        addLegend(position = "bottomleft", # Add a legend to the map
              pal = mypal_disp,            # Legend colors
              values = zlim_disp,          # Legend values
              title = disp_label,          # Legend title
              opacity = 0.7                # Legend opacity
             ) %>%
        addDrawToolbar(       # add toolbar
              polylineOptions     = TRUE,   # Enable polyline draw
              polygonOptions      = FALSE,
              circleOptions       = FALSE,
              circleMarkerOptions = FALSE,
              markerOptions       = FALSE,
              rectangleOptions    = FALSE,
             ) %>%
        addReverseSearchOSM(  # add location to polyline nodes
              showSearchLocation = TRUE,
                     showFeature = FALSE,
                     fitBounds   = FALSE,
                     displayText = FALSE
             )

To visualize the map, generate an HTML file in your files folder, download it to your computer, and open it in a browser. On the map, you can observe the evolution over time of the main features previously mentioned.

In [ ]:
### 6.5 export as a web page
saveWidget(p_ts, "Timeseries.html")


#<a name="Section 6"><hr color="#3B8A4C"><font size="6" color="#3B8A4C">**7. Timeseries cross-section**</font><hr color="#3B8A4C"></a>

In this section, we will create a line in the map on which we will project the values of the selected time series, allowing us to graph the evolution of deformation over time in a cross-section. In some cases, we could observe patterns caused by differential deformation that could reveal faults or geological contacts.

In [ ]:
#'##############
### 7. TIMESERIES CROSS-SECTION
#'##############

#### <a name="Section 3.1"><font size="5" color="#1F497D">Projecting raster values onto a line</font></a>

Define coordinates of a polyline, you can use the mean-velocity or timeseries maps to draw a line and get coordinates.

In [ ]:
#' 7.1. Create the cross-section to project the timeseries displacements

#'   Use option of draw a polyline in the dynamic map to get markers defining the cross-section coordinates

#    cross-section coords and spatvector format
     line_coords <- rbind(c(11.58334, 44.52499), c(11.08664, 44.63011))

#    geographic projection
     ts_line <- vect(line_coords, "lines", crs = "EPSG:4326")


Project the values from the raster dataset onto the defined line.

In [ ]:
# 7.2. Extract displacement values from each raster cell that the cross-section intersects
     disp <- extract(egms_ts_rd, ts_line, ID = FALSE, xy = TRUE)   # use the lonlat raster

#    split disp file in coordinates and values
     disp_values = subset(disp, select = -c(x, y))  # subset all columns except x and y
     disp_coord  = subset(disp, select =  c(x, y))  # subset only  x and y columns
#    round coordinates to 5 decimals
     disp_coord = round(disp_coord, 5)

#### <a name="Section 3.1"><font size="5" color="#1F497D">Plotting the cross-section</font></a>

To plot the displacement cross-section, we calculate the length of the line and extract the positions of the raster cells intersected by the line. Then, we compile a file of distances and displacements.

In [ ]:
# 7.3. Plot the cross-section

#      Calculate the total length of the line
       line_length <- perim(ts_line)

#      Create a sequence of distances along the line using the line length and the number of intersected cells
       distant <- seq(from = 0, to = line_length, length.out = nrow(disp))
#      round distant to 0 decimals
       distant = round(distant, 0)

#      Create a data frame with the distances and displacement values
       cross_sect <- data.frame(distance = distant, disp_values)
       dim(cross_sect)

[1] 127  32

Stack the time series into a long format with a column for the date that will be used for coloring

In [ ]:
#      Reshape the data frame to a long format using the pivot_longer function
#      This simplifies the process of plotting time series with differentiated colors
#      by concatenating all displacement columns and assigning labels for each date
       cross_sect_long = pivot_longer(cross_sect, cols = -1, names_to = "date", values_to = "displacement")

#      Remove 'X' character from date column
       cross_sect_long$date <- gsub("X", "", cross_sect_long$date)


In [ ]:
#    Create the plot
     ggplot(cross_sect_long, aes(distance, displacement, color = date)) +
            geom_line() +
            theme(legend.position = "bottom") +
            xlab("distance (m)") +
            ylab(disp_label)

The travel direction of the line does not follow the order in which you have entered the markers; it follows the GIS convention of the first quadrant, therefore the start of the line is the furthest to the northwest. Let's get the coordinates for the start and the end of the cross-section.

In [ ]:
# Attention!
#    The line is plotted using the first quadrant convention in GIS, where the origin (0, 0) is located at the most northwestern point.

#    Values and coordinates of the start and end of the cross-section
cat(
  "Distance", head(cross_sect$distance, 1), ", coordinates", paste(head(disp_coord, 1), collapse=" "),
  "\nDistance", tail(cross_sect$distance, 1), ", coordinates", paste(tail(disp_coord, 1), collapse=" ")
     )


Distance 0 , coordinates 11.08669 44.62845 
Distance 41143 , coordinates 11.58149 44.52379

Mapping the line to explore features derive from the cross-section graph

In [ ]:
# Plot the last line over the map
p_vel_pts_line = p_vel %>% lines(ts_line, col="white", lwd=2)

### export as a web page
saveWidget(p_vel_pts_line, "mean_velocity_points_line.html")

#<a name="Section 6"><hr color="#3B8A4C"><font size="6" color="#3B8A4C">**8. Export results**</font><hr color="#3B8A4C"></a>

In [ ]:
#'##############
### 8. EXPORT
#'##############

# Write the line to kml
writeVector(ts_line, filename = "ts_line.kml", filetype = "KML", overwrite=TRUE)

# write raster to geotiff
writeRaster(egms_vel_rd, filename = "egms_vel_rd.tif", filetype = "GTiff", overwrite=TRUE)
writeRaster(egms_ts_rd, filename = "egms_ts_rd.tif", filetype = "GTiff", overwrite=TRUE)

#'##############
### END
#'##############
#

#<a name="Section 6"><hr color="#3B8A4C"><font size="6" color="#3B8A4C">**Conclusion**</font><hr color="#3B8A4C"></a>

In this exercise, we have conducted an analysis of ground deformation around Bologna. This case study has been used only to illustrate the operation of the code and it was not intended to derive scientific conclusions about deformation patterns and mechanisms. This would require an in-depth study of the geology and field work that have not been carried out. Therefore, the features observed in the time series due to differential deformation could reveal geological contacts or faults, but these would need to be confirmed with a more comprehensive study.





https://land.copernicus.eu/pan-european/european-ground-motion-service

https://www.esa.int/ESA_Multimedia/Images/2022/07/Subsidence_patterns_around_Bologna